In [1]:
import numpy as np

from keras.models import Model, Sequential
from keras.layers import Input, Dense, BatchNormalization, RepeatVector, Concatenate, Merge, Masking
from keras.layers import LSTM, GRU, Embedding, TimeDistributed, Bidirectional
from keras import backend as K
from keras import optimizers
from keras.models import load_model

/Users/pulkitmaloo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/pulkitmaloo/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
all_data = np.load('train_dev_test.npz')

In [3]:
train_encoder_output = all_data['train_encoder_output']
train_decoder_input = all_data['train_decoder_input']
train_decoder_target = all_data['train_decoder_target'][:,1:,:]
validation_encoder_output = all_data['validation_encoder_output']
validation_decoder_input = all_data['validation_decoder_input']
validation_decoder_target = all_data['validation_decoder_target'][:,1:,:]
test_encoder_output = all_data['test_encoder_output']
test_decoder_input = all_data['test_decoder_input']
test_decoder_target = all_data['test_decoder_target'][:,1:,:]

In [4]:
print("Train Encoder Output", train_encoder_output.shape)
print("Train Decoder Input", train_decoder_input.shape)
print("Train Decoder Target", train_decoder_target.shape)

Train Encoder Output (30000, 512)
Train Decoder Input (30000, 38)
Train Decoder Target (30000, 38, 2531)


In [5]:
from caption_utils import *
train_fns_list, dev_fns_list, test_fns_list = load_split_lists()

train_captions_raw, dev_captions_raw, test_captions_raw = get_caption_split()
vocab = create_vocab(train_captions_raw)
token2idx, idx2token = vocab_to_index(vocab)    
captions_data = (train_captions_raw.copy(), dev_captions_raw.copy(), test_captions_raw.copy())
train_captions, dev_captions, test_captions = process_captions(captions_data, token2idx)

In [6]:
encoder_model = load_model('saved_models/encoder_model.h5')
decoder_model = load_model('saved_models/decoder_model.h5')

/Users/pulkitmaloo/anaconda3/lib/python3.6/site-packages/keras/engine/topology.py:1539: UserWarning: The list of outputs passed to the model is redundant. All outputs should only appear once. Found: [<tf.Tensor 'img_embedding/Relu:0' shape=(?, 300) dtype=float32>, <tf.Tensor 'img_embedding/Relu:0' shape=(?, 300) dtype=float32>]
  ' Found: ' + str(self.outputs))
/Users/pulkitmaloo/anaconda3/lib/python3.6/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [7]:
def seq_to_sentence(sent):
    return ' '.join([idx2token[idx] for idx in sent])

In [8]:
def generate_seq(img_input, alpha=1.):
    if img_input.shape != (1, 512):
        img_input = img_input.reshape(1, 512)
    
    assert(img_input.shape == (1, 512))
    stop_condition = False
    decoded_sentence = []
    target_seq = np.array([token2idx['<bos>']]).reshape(1, 1)
    states_value = encoder_model.predict(img_input)
    
    neg_log_proba = 0.
    while not stop_condition:

        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        output_tokens = np.squeeze(output_tokens)
        
        sampled_token_index = int(np.argmax(output_tokens))
        neg_log_proba -= np.log(output_tokens[sampled_token_index])
        
        sampled_char = idx2token[sampled_token_index]

        decoded_sentence += [sampled_char]

        if (sampled_char == '<eos>' or len(decoded_sentence) > 30):
            stop_condition = True

        target_seq = np.array([sampled_token_index]).reshape(1, 1)

        states_value = [h, c]
    return neg_log_proba/len(decoded_sentence)**alpha, ' '.join(decoded_sentence)

In [67]:
def decoder_one_step(sent, states_value, beam_size=5, len_norm=True, alpha=1):
    """ 
    sent: [neg_log_prob, [1, ...]]
    states_value: [h, c]
    return list of sent
    """
    last_word_idx = sent[1][-1]
    
    assert last_word_idx != token2idx['<eos>']
    
    target_seq = np.array([last_word_idx]).reshape(1, 1)
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
    output_tokens = np.squeeze(output_tokens)
    
    predicted_sentences = []    
#     output_tokens_beam = np.argpartition(-output_tokens, beam_size+4)
    output_tokens_beam = np.argsort(-output_tokens)
    output_tokens_beam = list(filter(lambda x: x not in [0, 1, 3], output_tokens_beam))[: beam_size]
    
    for predict_idx in output_tokens_beam:
#         if predict_idx in [0, 1, 3]:
#             continue
        
        new_sent = sent[1] + [int(predict_idx)]                
        
        if len_norm:
            neg_log_prob = sent[0] * max(len(sent[1])-1, 1)**alpha - np.log(output_tokens[int(predict_idx)])
            neg_log_prob /= max(len(new_sent)-1, 1)**alpha
        else:
            neg_log_prob = sent[0] - np.log(output_tokens[int(predict_idx)])
            
        predicted_sentences.append([neg_log_prob, new_sent])
        
#     print("from", sent[0], seq_to_sentence(sent[1]))
#     print("predicting")
#     for s in predicted_sentences:
#         print(s[0], seq_to_sentence(s[1]))
    
    return (predicted_sentences, [h, c])
    

In [68]:
def beam_search(img_input, beam_size=5, max_length=20, len_norm=True, alpha=0.7):
    """throws an error on beam_size 1 when <unk> is produced"""
    if img_input.shape != (1, 512):
        img_input = img_input.reshape(1, 512)    
    assert(img_input.shape == (1, 512))
    states_value_initial = encoder_model.predict(img_input)
    
    beg_sent = [0., [token2idx['<bos>']]]
#     print(beg_sent)
    top_sentences, states_value = decoder_one_step(beg_sent, states_value_initial, beam_size, len_norm, alpha)
#     print(list(map(lambda x: seq_to_sentence(x[1]), top_sentences)))
    
    stop_condition = False
    
    while not stop_condition:
        new_top_sentences = []
        for sent in top_sentences:
            if sent[1][-1] == token2idx['<eos>']:
                new_top_sentences += [sent]
                continue
            predicted_sent, new_states_value = decoder_one_step(sent, states_value, beam_size, len_norm, alpha)
            new_top_sentences += predicted_sent
            
        top_sentences = sorted(new_top_sentences, key=lambda x: x[0])[: beam_size]
        assert len(top_sentences) == beam_size, "Top Sentences"+str(len(top_sentences))+' '+str(beam_size)
        
        states_value = new_states_value

#         print(seq_to_sentence(top_sentences[0][1]))
        
        # Update stop condition
        eos_cnt = 0
        any_max_len = False
        for sent in top_sentences:
            if sent[1][-1] == token2idx['<eos>']:
                eos_cnt += 1
            if len(sent[1]) >= max_length:
                any_max_len = True
                print('Max len reached')
                break
        
        if any_max_len or (eos_cnt == beam_size):
            stop_condition = True        
            
    return top_sentences

In [69]:
for i in range(1, 100, 5):
    print("Original\n", seq_to_sentence(np.argmax(train_decoder_target[i, :], -1)))
    print("Greedy\n", generate_seq(train_encoder_output[i, :], alpha=0.5))
    print("Beam Search")
    top_sentences = beam_search(train_encoder_output[i, :], beam_size=5, max_length=30, alpha=0.5)
    for sent in top_sentences:
        print(sent[0], seq_to_sentence(sent[1][1: ]))
    print('*' * 25)

Original
 black dog chasing brown dog through snow <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Greedy
 (2.094863992301013, 'two dogs are playing in the sand <eos>')
Beam Search
2.7915681673634425 two dogs are playing in the sand <eos>
2.8118820178220227 two dogs playing in the sand <eos>
2.8491882935377317 two dogs are playing in the beach <eos>
3.2483585460552358 two dogs are playing in the side by a fence <eos>
3.2654059832327516 two dogs are playing in the side of a fence <eos>
*************************
Original
 a little girl plays croquet next to a truck <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Greedy
 (4.717805080947383, 'a young girl in a red shirt and jeans is playing on the street <eos>')
Beam Search
3.4117

In [29]:
[1,2,3][:5]

[1, 2, 3]